<a href="https://colab.research.google.com/github/keshav-b/PyTorch/blob/master/Sequence%20Models/Bi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

In [0]:
vocab = """
And who are you
The proud lord said
That I must bow so low
Only a cat of a different coat
That's all the truth I know

In a coat of gold or a coat of red
A lion still has claws
And mine are long and sharp, my lord
As long and sharp as yours


And so he spoke, and so he spoke
That lord of Castamere
But now the rains weep o'er his hall
With no one there to hear
Yes, now the rains weep o'er his hall
And not a soul to hear
"""

In [0]:
vocab = vocab.lower()

In [0]:
word_dict = {w: i for i, w in enumerate(list(set(vocab.split())))}
number_dict = {i: w for i, w in enumerate(list(set(vocab.split())))}

In [6]:
word_dict

{'a': 23,
 'all': 7,
 'and': 52,
 'are': 0,
 'as': 46,
 'bow': 35,
 'but': 32,
 'castamere': 11,
 'cat': 16,
 'claws': 27,
 'coat': 1,
 'different': 40,
 'gold': 17,
 'hall': 12,
 'has': 19,
 'he': 8,
 'hear': 41,
 'his': 9,
 'i': 2,
 'in': 37,
 'know': 31,
 'lion': 3,
 'long': 50,
 'lord': 44,
 'low': 10,
 'mine': 20,
 'must': 45,
 'my': 4,
 'no': 49,
 'not': 13,
 'now': 33,
 "o'er": 29,
 'of': 28,
 'one': 25,
 'only': 42,
 'or': 18,
 'proud': 53,
 'rains': 48,
 'red': 57,
 'said': 38,
 'sharp': 21,
 'sharp,': 14,
 'so': 54,
 'soul': 26,
 'spoke': 6,
 'spoke,': 43,
 'still': 5,
 'that': 56,
 "that's": 24,
 'the': 15,
 'there': 51,
 'to': 36,
 'truth': 39,
 'weep': 55,
 'who': 30,
 'with': 22,
 'yes,': 34,
 'you': 58,
 'yours': 47}

In [0]:
n_class = len(word_dict)
max_len = len(vocab.split())
n_hidden = 5

In [0]:
def make_batch(vocab):
    input_batch = []
    target_batch = []
    
    words = vocab.split()
    for i, word in enumerate(words[:-1]):
        input = [word_dict[n] for n in words[:(i+1)]]
        input = input + [0] *(max_len - len(input))
        
        target = word_dict[words[i+1]]
        
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)
     
    return input_batch, target_batch



#  -----------------------------------------
#  | INPUT                | TARGET          |
#  -----------------------------------------
#  | word1                | word2           |
#  | word1 + 2            | word3           |
#  | word1 + 2 + 3        | word4           |
#              .
#              .
#              .
#  | word1 + 2 +.. |V|-1  | last word       |

In [0]:
input_batch, target_batch = make_batch(vocab)

input_batch = Variable(torch.Tensor(input_batch))
target_batch = Variable(torch.LongTensor(target_batch))

In [0]:
dtype = torch.FloatTensor

In [0]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, bidirectional=True)
        self.W = nn.Parameter(torch.randn([n_hidden * 2, n_class]).type(dtype))
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))

    def forward(self, X):
        input = X.transpose(0, 1)  # input : [n_step, batch_size, n_class]

        hidden_state = Variable(torch.zeros(1*2, len(X), n_hidden))   # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        cell_state = Variable(torch.zeros(1*2, len(X), n_hidden))     # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]

        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state))
        outputs = outputs[-1]  # [batch_size, n_hidden]
        model = torch.mm(outputs, self.W) + self.b  # model : [batch_size, n_class]
        return model

In [0]:
model = Network()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
epochs = 10000

for epoch in range(1, epochs+1):
    
    optimizer.zero_grad()
    
    output = model(input_batch)
    
    loss = criterion(output, target_batch)
    if epoch % 500 == 0:
        print(f"epoch: {epoch} ===> loss:{loss}")

    loss.backward()
    optimizer.step()

epoch: 500 ===> loss:3.7432494163513184
epoch: 1000 ===> loss:3.464881658554077
epoch: 1500 ===> loss:3.467609167098999
epoch: 2000 ===> loss:3.3478853702545166
epoch: 2500 ===> loss:3.1607887744903564
epoch: 3000 ===> loss:3.170344352722168
epoch: 3500 ===> loss:3.047229051589966
epoch: 4000 ===> loss:2.997220277786255
epoch: 4500 ===> loss:2.9603426456451416
epoch: 5000 ===> loss:2.9192965030670166
epoch: 5500 ===> loss:2.8676185607910156
epoch: 6000 ===> loss:2.7643117904663086
epoch: 6500 ===> loss:2.1128740310668945
epoch: 7000 ===> loss:1.912989616394043
epoch: 7500 ===> loss:1.7553372383117676
epoch: 8000 ===> loss:1.631020426750183
epoch: 8500 ===> loss:2.3829903602600098
epoch: 9000 ===> loss:2.0513007640838623
epoch: 9500 ===> loss:1.84079110622406
epoch: 10000 ===> loss:1.7131637334823608


In [0]:
test_data = "the lord has sharp red claws"

In [0]:
test_data = test_data.lower()

In [0]:
test_input_batch = []


words = test_data.split()
for i, word in enumerate(words[:-1]):
    input = [word_dict[n] for n in words[:(i+1)]]
    input = input + [0] *(max_len - len(input))
    
    test_input_batch.append(np.eye(n_class)[input])

test_input_batch = Variable(torch.Tensor(test_input_batch))    

In [23]:
predict = model(test_input_batch).data.max(1, keepdim=True)[1]
predict

tensor([[28],
        [50],
        [28],
        [23],
        [23]])

In [28]:
print(test_data," --> ", [number_dict[n.item()] for n in predict.squeeze()])

the lord has sharp red claws  -->  ['of', 'long', 'of', 'a', 'a']
